<a href="https://colab.research.google.com/github/jiyoung37/Plant-disease-recognition/blob/main/Model_evaluation_testdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import

**1.1 Import kagglehub and download dataset**

In [ ]:
!pip install kagglehub
import kagglehub

# Download the latest version of the dataset
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

100%|██████████| 2.70G/2.70G [02:06<00:00, 22.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2


**1.2 Define the paths**

In [ ]:
import os

# Define the dataset path
root_folder = "/root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2"

data_dir = os.path.join(root_folder, 'new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/')

test_path = os.path.join(root_folder, 'test/test')
train_path = os.path.join(data_dir, 'train')
valid_path = os.path.join(data_dir, 'valid')

# Count the number of folders in a directory
def count_folders(path):
    if os.path.exists(path):
        return len([d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))])
    else:
        print(f"Path does not exist: {path}")
        return 0

# Count the number of files in a directory
def count_files(path):
    if os.path.exists(path):
        return len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])
    else:
        print(f"Path does not exist: {path}")
        return 0

# Function to count files in each folder of a given directory
def count_files_in_folders(path):
    if not os.path.exists(path):
        print(f"Path does not exist: {path}")
        return {}

    folder_file_counts = {}
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        if os.path.isdir(folder_path):
            file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
            folder_file_counts[folder] = file_count

    return folder_file_counts

# Print counts
print(f"Number of folders in train_path: {count_folders(train_path)}")
print(f"Number of folders in valid_path: {count_folders(valid_path)}")
print(f"Number of files in test_path: {count_files(test_path)}")
print('-------------------------------------------------------------------')
# Count files in each folder
train_folder_counts = count_files_in_folders(train_path)
valid_folder_counts = count_files_in_folders(valid_path)
# Display results
print("Files in each folder (train_path):")
for folder, count in train_folder_counts.items():
    print(f"{folder}: {count} files")

print("\nFiles in each folder (valid_path):")
for folder, count in valid_folder_counts.items():
    print(f"{folder}: {count} files")

Number of folders in train_path: 38
Number of folders in valid_path: 38
Number of files in test_path: 33
-------------------------------------------------------------------
Files in each folder (train_path):
Tomato___healthy: 1926 files
Strawberry___Leaf_scorch: 1774 files
Tomato___Target_Spot: 1827 files
Corn_(maize)___Northern_Leaf_Blight: 1908 files
Peach___healthy: 1728 files
Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 1722 files
Potato___healthy: 1824 files
Apple___Black_rot: 1987 files
Tomato___Bacterial_spot: 1702 files
Tomato___Late_blight: 1851 files
Grape___healthy: 1692 files
Tomato___Tomato_Yellow_Leaf_Curl_Virus: 1961 files
Tomato___Leaf_Mold: 1882 files
Cherry_(including_sour)___healthy: 1826 files
Apple___Cedar_apple_rust: 1760 files
Tomato___Tomato_mosaic_virus: 1790 files
Grape___Black_rot: 1888 files
Peach___Bacterial_spot: 1838 files
Tomato___Spider_mites Two-spotted_spider_mite: 1741 files
Corn_(maize)___Common_rust_: 1907 files
Raspberry___healthy: 1781 files
Potat

# 2. Importing necessary libraries for Modeling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import os

import tensorflow as tf
# Check GPU availability
if tf.config.list_physical_devices('GPU'):
    print("OK! GPU is available and ready for use!")
else:
    print("NO! GPU not found. Falling back to CPU.")

import PIL
import scipy

print(PIL.__version__)
print(scipy.__version__)

OK! GPU is available and ready for use!
11.1.0
1.13.1


Image generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50
lr = 0.0001

# Define paths
train_path = train_path
valid_path = valid_path

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
valid_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True)

val_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


Import extra test files form github

In [ ]:
!git clone https://github.com/DataScientest-Studio/oct24_bds_int_plant_recognition.git


Cloning into 'oct24_bds_int_plant_recognition'...
remote: Enumerating objects: 1321, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 1321 (delta 106), reused 101 (delta 62), pack-reused 1158 (from 2)
Receiving objects: 100% (1321/1321), 262.99 MiB | 17.34 MiB/s, done.
Resolving deltas: 100% (420/420), done.


In [ ]:
#%cd oct24_bds_int_plant_recognition/src/test_dataset_extra
import os

# Define the folder path
test_path = '/content/oct24_bds_int_plant_recognition/src/test_dataset_extra'


# Evaluation on test folder_1

Load model from google drive

In [ ]:
from tensorflow.keras.models import load_model
from google.colab import drive
drive.mount('/content/drive')

save_dir = "/content/drive/MyDrive/DataScientist_Projects/1_Plant_Recognition_Project/Model-evaluation"

Model_name = 'TL_MobileNetV2_unfrozen_lr_E-3'

model_path = f"{save_dir}/{Model_name}.keras"

# Load the model
model = load_model(model_path)
print("Model loaded successfully!")

Mounted at /content/drive
Model loaded successfully!


Evaluation on the test folder

In [ ]:
import os
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
import re

# Define helper function to preprocess a single image
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)  # Load image and resize
    img_array = img_to_array(img) / 255.0               # Convert to array and rescale
    return img_array

# Load and preprocess all images from the test folder
def load_test_images(test_path, target_size):
    test_images = []
    filenames = []
    for file in os.listdir(test_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Supported file formats
            file_path = os.path.join(test_path, file)
            test_images.append(preprocess_image(file_path, target_size))
            filenames.append(file)  # Save filenames for reference
    return np.array(test_images), filenames

# Function to check if prediction is correct based on common words
def is_prediction_correct(file_name, predicted_label):
    # Extract meaningful words from file name (split camel case or concatenated words)
    file_name = re.sub(r'\.\w+$', '', file_name)  # Remove file extension
    file_words = set(word.lower() for word in re.findall(r'[A-Z][a-z]*|[a-z]+', file_name))  # Handle camel case and lowercase

    # Extract meaningful words from predicted label
    label_words = set(word.lower() for word in re.findall(r'[A-Za-z]+', predicted_label))  # Convert to lowercase

    # Check if all file words are in the label words
    is_subset = file_words.issubset(label_words)

    # Debugging output to verify
    #print(f"File Words: {file_words}, Label Words: {label_words}, Is Subset: {is_subset}")

    return is_subset

# Parameters
IMG_SIZE = (256,256)  # Adjust to model's input size
test_path = test_path  # Define the test folder path

# Load and preprocess test images
test_images, filenames = load_test_images(test_path, target_size=IMG_SIZE)

# Make Predictions
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Map predictions to class labels
class_labels = train_generator.class_indices
class_indices = {v: k for k, v in class_labels.items()}  # Reverse mapping of class indices
predicted_labels = [class_indices[idx] for idx in predicted_classes]

# Process and evaluate predictions
results = []

for file, pred, label in zip(filenames, predictions, predicted_labels):
    confidence = np.max(pred)  # Get the highest probability (confidence score)
    is_correct = is_prediction_correct(file, label)  # Check prediction correctness
    results.append({"True class": file, "prediction": label, "confidence": confidence, "Correct": int(is_correct)})
    print(f"True class: {file}, Prediction: {label}, Confidence: {confidence:.2f}, Correct: {is_correct}")

# Summary of results
correct_predictions = sum(result['Correct'] for result in results)
total_predictions = len(results)
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
print(f"Total Predictions: {total_predictions}, Correct: {correct_predictions}, Accuracy: {accuracy:.2%}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step
True class: TomatoHealthy4.JPG, Prediction: Tomato___healthy, Confidence: 1.00, Correct: True
True class: AppleCedarRust1.JPG, Prediction: Apple___Cedar_apple_rust, Confidence: 1.00, Correct: True
True class: AppleCedarRust4.JPG, Prediction: Apple___Cedar_apple_rust, Confidence: 1.00, Correct: True
True class: AppleScab2.JPG, Prediction: Apple___Apple_scab, Confidence: 1.00, Correct: True
True class: PotatoEarlyBlight1.JPG, Prediction: Potato___Early_blight, Confidence: 1.00, Correct: True
True class: TomatoHealthy1.JPG, Prediction: Tomato___healthy, Confidence: 1.00, Correct: True
True class: TomatoEarlyBlight5.JPG, Prediction: Tomato___Early_blight, Confidence: 1.00, Correct: True
True class: PotatoHealthy1.JPG, Prediction: Potato___healthy, Confidence: 1.00, Correct: True
True class: TomatoYellowCurlVirus6.JPG, Prediction: Tomato___Tomato_Yellow_Leaf_Curl_Virus, Confidence: 1.00, Correct: True
True class: TomatoEarlyBlight6.JPG, Prediction: Tomat

Create DataFrame and save as CSV in google drive

In [ ]:
# Create a DataFrame from the results
df_results = pd.DataFrame(results)

# Print the DataFrame
print("\nSummary of Predictions and Feedback:")
display(df_results)

# Save DataFrame to CSV
output_csv_path = f"{save_dir}/{Model_name}_predictions_1.csv"  # Specify the output file path
df_results.to_csv(output_csv_path, index=False)

print(f"Predictions have been saved to {output_csv_path}")


Summary of Predictions and Feedback:


,True class,prediction,confidence,Correct
0,TomatoHealthy4.JPG,Tomato___healthy,1.000000,1
1,AppleCedarRust1.JPG,Apple___Cedar_apple_rust,1.000000,1
2,AppleCedarRust4.JPG,Apple___Cedar_apple_rust,1.000000,1
3,AppleScab2.JPG,Apple___Apple_scab,1.000000,1
4,PotatoEarlyBlight1.JPG,Potato___Early_blight,1.000000,1
5,TomatoHealthy1.JPG,Tomato___healthy,1.000000,1
6,TomatoEarlyBlight5.JPG,Tomato___Early_blight,0.999999,1
7,PotatoHealthy1.JPG,Potato___healthy,1.000000,1
8,TomatoYellowCurlVirus6.JPG,Tomato___Tomato_Yellow_Leaf_Curl_Virus,1.000000,1
9,TomatoEarlyBlight6.JPG,Tomato___Early_blight,1.000000,1


Predictions have been saved to /content/drive/MyDrive/DataScientist_Projects/1_Plant_Recognition_Project/Model-evaluation/TL_MobileNetV2_unfrozen_lr_E-3_predictions_1.csv


Prediction accuracy

In [ ]:
df_results["Correct"].value_counts(normalize=True)

,proportion
Correct,
1,1.0


# Evaluation on test folder_2

Load model from Github

In [ ]:
# Remove the existing directory
!rm -rf oct24_bds_int_plant_recognition
!git clone https://github.com/DataScientest-Studio/oct24_bds_int_plant_recognition.git

test_path = '/content/oct24_bds_int_plant_recognition/src/test_dataset_extra_283'
#!ls /content/oct24_bds_int_plant_recognition/src/test_dataset_extra_

Cloning into 'oct24_bds_int_plant_recognition'...
remote: Enumerating objects: 1321, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 1321 (delta 106), reused 101 (delta 62), pack-reused 1158 (from 2)
Receiving objects: 100% (1321/1321), 262.99 MiB | 17.23 MiB/s, done.
Resolving deltas: 100% (420/420), done.


Evaluation on the test folder

In [ ]:
import os
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
import re

# Define helper function to preprocess a single image
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)  # Load image and resize
    img_array = img_to_array(img) / 255.0               # Convert to array and rescale
    return img_array

# Load and preprocess all images from the test folder
def load_test_images(test_path, target_size):
    test_images = []
    filenames = []
    for file in os.listdir(test_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Supported file formats
            file_path = os.path.join(test_path, file)
            test_images.append(preprocess_image(file_path, target_size))
            filenames.append(file)  # Save filenames for reference
    return np.array(test_images), filenames

# Function to check if prediction is correct based on common words
def is_prediction_correct(file_name, predicted_label):
    # Extract meaningful words from file name (split camel case or concatenated words)
    file_name = re.sub(r'\.\w+$', '', file_name)  # Remove file extension
    file_words = set(word.lower() for word in re.findall(r'[A-Z][a-z]*|[a-z]+', file_name))  # Handle camel case and lowercase

    # Extract meaningful words from predicted label
    label_words = set(word.lower() for word in re.findall(r'[A-Za-z]+', predicted_label))  # Convert to lowercase

    # Check if all file words are in the label words
    is_subset = file_words.issubset(label_words)

    # Debugging output to verify
    #print(f"File Words: {file_words}, Label Words: {label_words}, Is Subset: {is_subset}")

    return is_subset

# Parameters
IMG_SIZE = IMG_SIZE  # Adjust to model's input size
test_path = test_path  # Define the test folder path

# Load and preprocess test images
test_images, filenames = load_test_images(test_path, target_size=IMG_SIZE)

# Make Predictions
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Map predictions to class labels
class_labels = train_generator.class_indices
class_indices = {v: k for k, v in class_labels.items()}  # Reverse mapping of class indices
predicted_labels = [class_indices[idx] for idx in predicted_classes]

# Process and evaluate predictions
results = []

for file, pred, label in zip(filenames, predictions, predicted_labels):
    confidence = np.max(pred)  # Get the highest probability (confidence score)
    is_correct = is_prediction_correct(file, label)  # Check prediction correctness
    results.append({"True class": file, "prediction": label, "confidence": confidence, "Correct": int(is_correct)})
    print(f"True class: {file}, Prediction: {label}, Confidence: {confidence:.2f}, Correct: {is_correct}")

# Summary of results
correct_predictions = sum(result['Correct'] for result in results)
total_predictions = len(results)
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
print(f"Total Predictions: {total_predictions}, Correct: {correct_predictions}, Accuracy: {accuracy:.2%}")


9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 573ms/step
True class: apple_scab_46.JPG, Prediction: Apple___Apple_scab, Confidence: 1.00, Correct: True
True class: tomato_yellow_leaf_curl_virus_30.JPG, Prediction: Tomato___Tomato_Yellow_Leaf_Curl_Virus, Confidence: 1.00, Correct: True
True class: tomato_mosaic_virus_38.JPG, Prediction: Tomato___Tomato_mosaic_virus, Confidence: 1.00, Correct: True
True class: tomato_mosaic_virus_11.JPG, Prediction: Tomato___Tomato_mosaic_virus, Confidence: 1.00, Correct: True
True class: tomato_yellow_leaf_curl_virus_46.JPG, Prediction: Tomato___Tomato_Yellow_Leaf_Curl_Virus, Confidence: 1.00, Correct: True
True class: tomato_mosaic_virus_16.JPG, Prediction: Tomato___Tomato_mosaic_virus, Confidence: 1.00, Correct: True
True class: apple_scab_08.JPG, Prediction: Apple___Apple_scab, Confidence: 1.00, Correct: True
True class: apple_scab_48.JPG, Prediction: Apple___Apple_scab, Confidence: 1.00, Correct: True
True class: apple_scab_07.JPG, Prediction: Apple___Apple_scab, Con

Create DataFrame and save as CSV in google drive

In [ ]:
# Create a DataFrame from the results
df_results = pd.DataFrame(results)

# Print the DataFrame
print("\nSummary of Predictions and Feedback:")
display(df_results)

# Save DataFrame to CSV
output_csv_path = f"{save_dir}/{Model_name}_predictions_2.csv"
df_results.to_csv(output_csv_path, index=False)

print(f"Predictions have been saved to {output_csv_path}")


Summary of Predictions and Feedback:


,True class,prediction,confidence,Correct
0,apple_scab_46.JPG,Apple___Apple_scab,1.0,1
1,tomato_yellow_leaf_curl_virus_30.JPG,Tomato___Tomato_Yellow_Leaf_Curl_Virus,1.0,1
2,tomato_mosaic_virus_38.JPG,Tomato___Tomato_mosaic_virus,1.0,1
3,tomato_mosaic_virus_11.JPG,Tomato___Tomato_mosaic_virus,1.0,1
4,tomato_yellow_leaf_curl_virus_46.JPG,Tomato___Tomato_Yellow_Leaf_Curl_Virus,1.0,1
...,...,...,...,...
278,tomato_yellow_leaf_curl_virus_10.JPG,Tomato___Tomato_Yellow_Leaf_Curl_Virus,1.0,1
279,AppleScab3.JPG,Apple___Apple_scab,1.0,1
280,strawberry_leaf_scorch_19.JPG,Strawberry___Leaf_scorch,1.0,1
281,tomato_target_spot_48.JPG,Tomato___Target_Spot,1.0,1


Predictions have been saved to /content/drive/MyDrive/DataScientist_Projects/1_Plant_Recognition_Project/Model-evaluation/TL_MobileNetV2_unfrozen_lr_E-3_predictions_2.csv


Prediction accuracy

In [ ]:
df_results["Correct"].value_counts(normalize=True)

,proportion
Correct,
1,1.0
